In [3]:
embeding_matrix = load_pickle_file("../../new_disk/embeding_matrix")
word2index = load_pickle_file("../../new_disk/word2index")
index_sequences = load_pickle_file("../../new_disk/index_sequences.pkl")
labels = load_pickle_file("./labels")
SEQUENCE_LENGTH = 434
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3000440

In [4]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)
labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Dataset split

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [6]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>LSTM</h1>

In [7]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [8]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [40]:
NUM_FILTERS = 8
KERNEL_SIZE = 12
HIDDEN_UNITS = 4
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
conv_model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
conv_model.add(embedding_layer)
# conv_model.add(SpatialDropout1D())
conv_model.add(Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.09)))
conv_model.add(Dropout(0.2))
# conv_model.add(BatchNormalization())
conv_model.add(MaxPool1D(pool_size = 15))
conv_model.add(GlobalMaxPool1D())
# conv_model.add(Flatten())
conv_model.add(Dense(2,activation='sigmoid'))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
# lstm_model.add(Dense(LSTM_HIDDEN_DIM_SIZE//4, activation = 'relu'))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//4, activation = 'tanh',dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))


In [31]:
conv_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 434, 300)          900132300 
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 423, 8)            28808     
_________________________________________________________________
dropout_5 (Dropout)          (None, 423, 8)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 28, 8)             0         
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 8)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 18        
Total params: 900,161,126
Trainable params: 28,826
Non-trainable params: 900,132,300
___________________________________

In [39]:
conv_model.reset_states()

In [12]:
opt = tk.optimizers.Adam(learning_rate=0.002)

In [41]:
conv_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
history = conv_model.fit(data_train,labels_train,epochs = 10,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 7380 samples, validate on 821 samples
Epoch 1/10
7380/7380 [==============================] - 8s 1ms/sample - loss: 0.9085 - acc: 0.6272 - val_loss: 0.7064 - val_acc: 0.8544
Epoch 2/10
7380/7380 [==============================] - 7s 926us/sample - loss: 0.6707 - acc: 0.8128 - val_loss: 0.6572 - val_acc: 0.7467
Epoch 3/10
7380/7380 [==============================] - 7s 1ms/sample - loss: 0.6377 - acc: 0.8850 - val_loss: 0.6331 - val_acc: 0.9531
Epoch 4/10
7380/7380 [==============================] - 7s 1ms/sample - loss: 0.6127 - acc: 0.9126 - val_loss: 0.6282 - val_acc: 0.9050
Epoch 5/10
7380/7380 [==============================] - 7s 1ms/sample - loss: 0.5946 - acc: 0.9174 - val_loss: 0.5943 - val_acc: 0.9629
Epoch 6/10
7380/7380 [==============================] - 7s 1ms/sample - loss: 0.5744 - acc: 0.9333 - val_loss: 0.5865 - val_acc: 0.9464
Epoch 7/10
7380/7380 [==============================] - 8s 1ms/sample - loss: 0.5631 - acc: 0.9260 - val_loss: 0.5658 - val_acc: 0.9756

In [46]:
history = conv_model.fit(data_train,labels_train,epochs = 2,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 7380 samples, validate on 821 samples
Epoch 1/2
7380/7380 [==============================] - 7s 927us/sample - loss: 0.5130 - acc: 0.9538 - val_loss: 0.5225 - val_acc: 0.9714
Epoch 2/2
7380/7380 [==============================] - 7s 994us/sample - loss: 0.5087 - acc: 0.9525 - val_loss: 0.5082 - val_acc: 0.9860


In [51]:
conv_model.save("./Checkpoints/conv_model.h5")

In [52]:
conv_model.save_weights("./Checkpoints/conv_weights")

In [47]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [48]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=0)

0.9850894632206759

In [49]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.985372988785958

In [50]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.9853011108574907

In [19]:
i=0
for seq in data_train[128:250]:
    for el in seq:
        if np.isnan(el):
            print(i)
                
#         if type(arr) is np.ndarray:
#             print(i)
    i+=1

In [2]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
